# Classificação dos personagens dos Simpsons

Alunos: Alexandre Borges Baccarini Júnior - 2515520
        Leonardo
        Mateus

## Objetivo

Desenvolver um sistema inteligente para classificação de personagens da série Os Simpsons utilizando os classificadores k-NN e SVM.


### Baixar as bibliotecas

In [ ]:
#RODAR UMA VEZ PARA BAIXAR AS BIBLIOTECAS

%pip install numpy pandas Pillow scikit-learn keras imblearn

### Extração de Características

In [11]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from keras.applications.vgg16 import VGG16, preprocess_input

def load_img(img_path, target_size):
    img = Image.open(img_path)
    img = img.convert('RGB')
    img = img.resize(target_size)
    return img

def extract_features(directory):
    model = VGG16(weights='imagenet', include_top=False)
    features = []
    labels = []
    for filename in os.listdir(directory):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            img_path = os.path.join(directory, filename)
            label = directory.split(os.sep)[-1]
            try:
                image = load_img(img_path, target_size=(224, 224))
                image = np.array(image)
                image = np.expand_dims(image, axis=0)
                image = preprocess_input(image)
                feature = model.predict(image).flatten()
                features.append(feature)
                labels.append(label)
            except Exception as e:
                print(f"Error processing image: {img_path}")
                print(f"Exception: {e}")
    return features, labels

# TREINO
bart_dir = "./imgs/Train/bart"
homer_dir = "./imgs/Train/homer"
lisa_dir = "./imgs/Train/lisa"
maggie_dir = "./imgs/Train/maggie"
marge_dir = "./imgs/Train/marge"

bart_features, bart_labels = extract_features(bart_dir)
homer_features, homer_labels = extract_features(homer_dir)
lisa_features, lisa_labels = extract_features(lisa_dir)
maggie_features, maggie_labels = extract_features(maggie_dir)
marge_features, marge_labels = extract_features(marge_dir)

all_features = bart_features + homer_features + lisa_features + maggie_features + marge_features
all_labels = bart_labels + homer_labels + lisa_labels + maggie_labels + marge_labels

data = {'class': all_labels}
for i in range(len(all_features[0])):
    data[f'feature_{i+1}'] = [feat[i] for feat in all_features]

df = pd.DataFrame(data)

csv_train_path = "./data/Train/feats.csv"
df.to_csv(csv_train_path, index=False)

# TESTE
bart_dir = "./imgs/Valid/bart"
homer_dir = "./imgs/Valid/homer"
lisa_dir = "./imgs/Valid/lisa"
maggie_dir = "./imgs/Valid/maggie"
marge_dir = "./imgs/Valid/marge"

bart_features, bart_labels = extract_features(bart_dir)
homer_features, homer_labels = extract_features(homer_dir)
lisa_features, lisa_labels = extract_features(lisa_dir)
maggie_features, maggie_labels = extract_features(maggie_dir)
marge_features, marge_labels = extract_features(marge_dir)

all_features = bart_features + homer_features + lisa_features + maggie_features + marge_features
all_labels = bart_labels + homer_labels + lisa_labels + maggie_labels + marge_labels

data = {'class': all_labels}
for i in range(len(all_features[0])):
    data[f'feature_{i+1}'] = [feat[i] for feat in all_features]

df = pd.DataFrame(data)

csv_test_path = "./data/Valid/feats.csv"
df.to_csv(csv_test_path, index=False)


2025-06-28 17:39:45.934567: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/tmp/ipykernel_4445/475261520.py:23: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  image = np.array(image)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 534ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 289ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 302ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 297ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 293ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 284ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 284ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 284ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 303ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 286ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 286ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 282ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 285ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 287ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 280ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 287ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 287ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 283ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 282ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 288ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 280ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 293ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 284ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 285ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 289ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

### Treinamento e Avaliação dos Modelos
#### k-NN

In [12]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import GridSearchCV
import pandas as pd

try:
    from imblearn.over_sampling import SMOTE
    smote_available = True
except ImportError:
    smote_available = False
    print("SMOTE não disponível - usando RandomOverSampler")
    from imblearn.over_sampling import RandomOverSampler

# Carrega os dados
csv_train_path = "./data/Train/feats.csv"
df_train = pd.read_csv(csv_train_path)

# Análise de distribuição
print("Distribuição das classes no treino:")
print(df_train['class'].value_counts())

# Separa os dados
X_train = df_train.drop('class', axis=1)
y_train = df_train['class']

# Codifica as classes
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)

# Normaliza os dados
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Balanceamento
if smote_available:
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train_encoded)
else:
    ros = RandomOverSampler(random_state=42)
    X_resampled, y_resampled = ros.fit_resample(X_train_scaled, y_train_encoded)

# Grid de parâmetros
valores_grid = [
    {'n_neighbors': [3, 5, 7, 9], 'metric': ['euclidean'], 'weights': ['distance']},
    {'n_neighbors': [3, 5, 7, 9], 'metric': ['manhattan'], 'weights': ['distance']}
]

# Cria e treina o modelo
modelo = KNeighborsClassifier()
gridKNN = GridSearchCV(estimator=modelo, param_grid=valores_grid, cv=5, verbose=1, n_jobs=-1, scoring='f1_macro')
gridKNN.fit(X_resampled, y_resampled)

# Resultados
print(f"\nMelhor acurácia: {gridKNN.best_score_}")
print(f"Melhor K: {gridKNN.best_estimator_.n_neighbors}")
print(f"Melhor distância: {gridKNN.best_estimator_.metric}")
print(f"Melhores pesos: {gridKNN.best_estimator_.weights}")

Distribuição das classes no treino:
class
bart      78
homer     61
lisa      33
maggie    30
marge     23
Name: count, dtype: int64
Fitting 5 folds for each of 8 candidates, totalling 40 fits

Melhor acurácia: 0.6035610595661784
Melhor K: 3
Melhor distância: euclidean
Melhores pesos: distance


In [13]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pandas as pd

# Carrega os dados de teste
csv_test_path = "./data/Valid/feats.csv"
df_test = pd.read_csv(csv_test_path)

# Análise de distribuição
print("\nDistribuição das classes no teste:")
print(df_test['class'].value_counts())

# Separa os dados
X_test = df_test.drop('class', axis=1)
y_test = df_test['class']

# Codifica as classes
y_test_encoded = le.transform(y_test)

# Normaliza com o mesmo scaler do treino
X_test_scaled = scaler.transform(X_test)

# Cria o modelo com melhores parâmetros
modelo = KNeighborsClassifier(
    n_neighbors=gridKNN.best_estimator_.n_neighbors,
    metric=gridKNN.best_estimator_.metric,
    weights=gridKNN.best_estimator_.weights
)

# Treina com dados resampleados
modelo.fit(X_resampled, y_resampled)

# Predição
y_pred_encoded = modelo.predict(X_test_scaled)

# Avaliação
print("\nAvaliação do Modelo:")
print(f"Acurácia: {accuracy_score(y_test_encoded, y_pred_encoded)}")
print("\nRelatório de Classificação:")
target_names = le.classes_
print(classification_report(y_test_encoded, y_pred_encoded, target_names=target_names))

print("\nMatriz de Confusão:")
print(confusion_matrix(y_test_encoded, y_pred_encoded))


Distribuição das classes no teste:
class
bart      35
homer     25
lisa      13
maggie    12
marge     10
Name: count, dtype: int64

Avaliação do Modelo:
Acurácia: 0.25263157894736843

Relatório de Classificação:
              precision    recall  f1-score   support

        bart       0.00      0.00      0.00        35
       homer       0.67      0.24      0.35        25
        lisa       0.27      0.54      0.36        13
      maggie       0.19      0.92      0.31        12
       marge       0.00      0.00      0.00        10

    accuracy                           0.25        95
   macro avg       0.22      0.34      0.20        95
weighted avg       0.24      0.25      0.18        95


Matriz de Confusão:
[[ 0  1 13 21  0]
 [ 0  6  2 16  1]
 [ 0  1  7  5  0]
 [ 0  0  1 11  0]
 [ 0  1  3  6  0]]


/home/alexandre/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/alexandre/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/alexandre/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


#### SVM

In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
import numpy as np

# Carrega os dados
df_train = pd.read_csv(csv_train_path)

# Separa os dados de treino e teste
X_train = df_train.drop('class', axis=1)
y_train = df_train['class']

# Normaliza os dados
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)

# Definindo os valores que serão testados
valores_C = np.array([0.001, 0.01, 0.1, 1, 10, 100])
valores_gamma = np.array([0.001, 0.01, 0.1, 1, 10, 100])
tipos_kernel = ['rbf', 'linear', 'poly', 'sigmoid']

valores_grid = {
    'C': valores_C,
    'gamma': valores_gamma,
    'kernel': tipos_kernel
}

# Criando o modelo
modelo = SVC()

# Criando os grids
gridSVM = GridSearchCV(estimator=modelo, param_grid=valores_grid, cv=5, verbose=1, n_jobs=-1)

# Treinando os grids
gridSVM.fit(X_train, y_train)

# Imprimindo os melhores parâmetros
print(f"Melhor acurácia: {gridSVM.best_score_}")
print(f"Melhor C: {gridSVM.best_estimator_.C}")
print(f"Melhor gamma: {gridSVM.best_estimator_.gamma}")
print(f"Melhor kernel: {gridSVM.best_estimator_.kernel}")

Fitting 5 folds for each of 144 candidates, totalling 720 fits


In [ ]:
from sklearn.metrics import accuracy_score, classification_report

# Carrega os dados de teste
df_test = pd.read_csv(csv_test_path)

X_test = df_test.drop('class', axis=1)
y_test = df_test['class']

# Normaliza os dados
scaler = MinMaxScaler()
X_test = scaler.fit_transform(X_test)

# Criando o modelo
modelo = SVC(C=gridSVM.best_estimator_.C, gamma=gridSVM.best_estimator_.gamma, kernel=gridSVM.best_estimator_.kernel)

# Treinando o modelo
modelo.fit(X_train, y_train)

# Testando o modelo
y_pred = modelo.predict(X_test)

# Avaliando o modelo

print(f"Acurácia: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred))

#### DECISION TREE

In [ ]:
# DESCOBRINDO OS MELHORES PARÂMETROS PARA O DECISION TREE

from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

# Carrega os dados
df_train = pd.read_csv(csv_train_path)

# Separa os dados de treino e teste
X_train = df_train.drop('class', axis=1)
y_train = df_train['class']

# Normaliza os dados
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)

# Definindo os valores que serão testados
valores_grid = {
    'criterion': np.array(['gini', 'entropy']),
    'splitter': np.array(['best', 'random']),
    'max_depth': np.array([10, 50, 100, 200, 500]),
    'min_samples_split': np.array([2, 5, 10, 20]),
    'min_samples_leaf': np.array([1, 2, 5, 10])
}

# Criando o modelo
modelo = DecisionTreeClassifier()

# Criando os grids
gridDT = GridSearchCV(estimator=modelo, param_grid=valores_grid, cv=5, verbose=1, n_jobs=-1)

# Treinando os grids
gridDT.fit(X_train, y_train)

# Imprimindo os melhores parâmetros
print(f"Melhor acurácia: {gridDT.best_score_}")
print(f"Melhor criterion: {gridDT.best_estimator_.criterion}")
print(f"Melhor splitter: {gridDT.best_estimator_.splitter}")
print(f"Melhor max_depth: {gridDT.best_estimator_.max_depth}")
print(f"Melhor min_samples_split: {gridDT.best_estimator_.min_samples_split}")
print(f"Melhor min_samples_leaf: {gridDT.best_estimator_.min_samples_leaf}")

In [ ]:
# TESTANDO O MODELO

from sklearn.metrics import accuracy_score, classification_report

# Carrega os dados de teste
df_test = pd.read_csv(csv_test_path)

X_test = df_test.drop('class', axis=1)
y_test = df_test['class']

# Normaliza os dados
scaler = MinMaxScaler()
X_test = scaler.fit_transform(X_test)

# Criando o modelo
modelo = DecisionTreeClassifier(criterion=gridDT.best_estimator_.criterion, splitter=gridDT.best_estimator_.splitter, max_depth=gridDT.best_estimator_.max_depth, min_samples_split=gridDT.best_estimator_.min_samples_split, min_samples_leaf=gridDT.best_estimator_.min_samples_leaf)

# Treinando o modelo
modelo.fit(X_train, y_train)

# Testando o modelo
y_pred = modelo.predict(X_test)

# Avaliando o modelo

print(f"Acurácia: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred))

### Comparação dos Modelos

In [ ]:
#comparação -code